In [1]:
import torch
import sys
print("GPU Available: ", torch.cuda.is_available())
import contractions
from datasets import load_dataset
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer,BertModel, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

GPU Available:  False


C:\Users\melih\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\melih\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [2]:
ds = load_dataset("Annanay/aml_song_lyrics_balanced")
train_dataset = load_dataset("Annanay/aml_song_lyrics_balanced", split="train")
train_df = train_dataset.to_pandas()

test_dataset  = load_dataset("Annanay/aml_song_lyrics_balanced", split="test")
test_df = test_dataset.to_pandas()

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\melih\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\melih\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\melih\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Define a basic list of common English stopwords
stop_words = { 'a', 'an', 'the', 'and', 'or', 'nor', 'so', 'yet', 'for', 'in', 'on', 'at', 'by',
              'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',
              'above', 'below', 'to', 'from', 'up', 'down', 'out', 'over', 'under', 'again', 'further',
              'is', 'am', 'are','was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'do', 'does', 'did', 'doing',
              'will', 'would', 'shall', 'should', 'can', 'could', 'may', 'might', 'must', 'who', 'whom',
              'whose', 'which', 'that', 'this', 'these', 'those', 'any', 'some', 'all', 'both',
              'each', 'every', 'either', 'neither', 'another', 'such', 'up', 'down', 'out', 'off',
              'over', 'under'}

In [4]:
# Define a function for text preprocessing
def preprocess_lyrics(lyrics):
    lyrics = contractions.fix(lyrics)
    # Remove anything in brackets (like [Verse 1])
    lyrics = re.sub(r'\[.*?\]', '', lyrics)
    # Replace literal '\n' with spaces
    lyrics = lyrics.replace('\\n', ' ')
    # Remove any isolated 'n' caused by incorrect newline handling
    lyrics = re.sub(r'\bn\b', '', lyrics)
    # Remove non-alphabetic characters
    # lyrics = re.sub(r'[^a-zA-Z\s]', '', lyrics)
    # Convert to lowercase
    lyrics = lyrics.lower()
    # Tokenize the lyrics
    tokens = word_tokenize(lyrics)
    print()
    # Remove stopwords and lemmatize the tokens
    cleaned_lyrics = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(cleaned_lyrics)
def minimal_cleaning(lyrics):
    # Expand contractions (e.g., "can't" -> "cannot")
    lyrics = contractions.fix(lyrics)
    # Remove anything in brackets (like [Verse 1])
    lyrics = re.sub(r'\[.*?\]', '', lyrics)
    # Replace literal '\n' with spaces
    lyrics = lyrics.replace('\\n', ' ')
    # Remove any isolated 'n' caused by incorrect newline handling
    lyrics = re.sub(r'\bn\b', '', lyrics)
    # Convert to lowercase (optional, depending on your model)
    lyrics = lyrics.lower()
    return lyrics

def advanced_processing(lyrics):
    # Tokenize the lyrics
    tokens = word_tokenize(lyrics)
    # Remove stopwords and lemmatize the tokens
    cleaned_lyrics = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(cleaned_lyrics)

In [5]:
cleaned_lyrics = [minimal_cleaning(lyric) for lyric in train_df['lyrics']]
train_df['cleaned_lyrics'] = cleaned_lyrics

cleaned_lyrics_test = [minimal_cleaning(lyric) for lyric in test_df['lyrics']]
test_df['cleaned_lyrics'] = cleaned_lyrics_test

# Encode the target variable (mood)
label_encoder = LabelEncoder()
train_df['mood_encoded'] = label_encoder.fit_transform(train_df['mood'])
test_df['mood_encoded'] = label_encoder.fit_transform(test_df['mood'])

train_dataset = Dataset.from_pandas(train_df[['cleaned_lyrics', 'mood_encoded']])
test_dataset = Dataset.from_pandas(test_df[['cleaned_lyrics', 'mood_encoded']])

In [6]:
# # Load DistilBERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Tokenize dataset
# def tokenize_function(examples):
#     return tokenizer(examples['cleaned_lyrics'], padding='max_length', truncation=True, max_length=256)

# train_dataset = train_dataset.map(tokenize_function, batched=True)
# test_dataset = test_dataset.map(tokenize_function, batched=True)

# train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'mood_encoded'])
# test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'mood_encoded'])

# train_dataset = train_dataset.rename_column("mood_encoded", "labels")
# test_dataset = test_dataset.rename_column("mood_encoded", "labels")

# # Load DistilBERT model and move to GPU
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_)).to('cuda')

In [7]:
# # Training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy='epoch',
#     save_strategy='epoch',
#     learning_rate=2e-5,  
#     per_device_train_batch_size=16,  
#     per_device_eval_batch_size=16,
#     num_train_epochs=2,  
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
#     save_total_limit=1,
#     logging_dir='./logs',
#     logging_steps=10,
#     report_to="tensorboard",
# )

# # Trainer setup
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
# )

# # Fine-tune the model
# trainer.train()

In [8]:
# # Evaluate the model
# results = trainer.evaluate()

# print(results)

# # Get predictions
# predictions = trainer.predict(test_dataset)

# # Extract predicted labels
# preds = predictions.predictions.argmax(-1)

# # Calculate accuracy, precision, recall, and F1-score
# accuracy = accuracy_score(test_dataset['labels'], preds)
# precision, recall, f1, _ = precision_recall_fscore_support(test_dataset['labels'], preds, average='weighted')

# print(f'Test Accuracy: {accuracy:.4f}')
# print(f'Test Precision: {precision:.4f}')
# print(f'Test Recall: {recall:.4f}')
# print(f'Test F1-score: {f1:.4f}')

# # Get predictions
# predictions = trainer.predict(train_dataset)

# # Extract predicted labels
# preds = predictions.predictions.argmax(-1)

# # Calculate accuracy, precision, recall, and F1-score
# accuracy = accuracy_score(train_dataset['labels'], preds)
# precision, recall, f1, _ = precision_recall_fscore_support(train_dataset['labels'], preds, average='weighted')

# print(f'Train Accuracy: {accuracy:.4f}')
# print(f'Train Precision: {precision:.4f}')
# print(f'Train Recall: {recall:.4f}')
# print(f'Train F1-score: {f1:.4f}')

In [6]:
data=pd.read_csv('D:/Python_Projects/spotivibe_exp/data_with_cleaned_lyrics.csv')

In [15]:
# from sklearn.feature_extraction.text import CountVectorizer
# import pandas as pd

# # Create CountVectorizer for unigrams
# unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))

# # Fit the vectorizer on the training data (this keeps the matrix sparse)
# X_train_unigram = unigram_vectorizer.fit_transform(X_train)

# # Convert to DataFrame but keep the sparse format
# df_unigram = pd.DataFrame.sparse.from_spmatrix(X_train_unigram, columns=unigram_vectorizer.get_feature_names_out())

# # Add labels to the DataFrame (y_train contains the mood categories)
# df_unigram['label'] = y_train.values

# # Function to count unique unigrams per label (optimized for sparse DataFrame)
# def count_unique_ngrams_sparse(df):
#     unique_ngrams_per_label = {}
#     for label in df['label'].unique():
#         # Select only rows belonging to the current label
#         df_label = df[df['label'] == label]
#         # Drop the label column to focus only on the features
#         df_label = df_label.drop(columns=['label'])
#         # Count non-zero (i.e., present) features for each label (optimized for sparse DataFrame)
#         unique_ngrams_per_label[label] = df_label.astype(bool).sum(axis=1).sum()
#     return unique_ngrams_per_label

# # Count unique unigrams per label using sparse matrices
# unigram_counts = count_unique_ngrams_sparse(df_unigram)

# # Print the results
# print("Unique Unigram Counts per Label:", unigram_counts)
# #OUTPUT: Unique Unigram Counts per Label: {1: 291334, 2: 244674, 0: 303024, 3: 319309}

In [8]:
# X_train, X_test, y_train, y_test = train_test_split(data['cleaned_lyrics'], data['mood_cats'], test_size=0.2, random_state=42)
# # Combine unigrams and bigrams in TF-IDF vectorizer
# vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=768)
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf = vectorizer.transform(X_test)

# # Load pre-trained BERT tokenizer and model, and move the model to the GPU
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained('bert-base-uncased').to('cuda')

# # Function to extract BERT embeddings with CUDA
# def extract_bert_embeddings(text_data):
#     # Tokenize and encode the input text, and move inputs to the GPU
#     inputs = tokenizer(text_data, return_tensors='pt', padding=True, truncation=True, max_length=512).to('cuda')
    
#     # Get the embeddings from the BERT model
#     with torch.no_grad():
#         outputs = bert_model(**inputs)
    
#     # Move the outputs back to the CPU (if necessary) and return the [CLS] token embedding
#     return outputs.last_hidden_state[:, 0, :].cpu().numpy()

# # Extract BERT embeddings for training and test sets, leveraging CUDA
# X_train_bert = [extract_bert_embeddings(text) for text in X_train]
# X_test_bert = [extract_bert_embeddings(text) for text in X_test]

C:\Users\melih\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\melih\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [9]:
# from scipy.sparse import hstack

# # Convert TF-IDF features to dense format (to match BERT embeddings)
# X_train_tfidf_dense = X_train_tfidf.toarray()
# X_test_tfidf_dense = X_test_tfidf.toarray()

# # Convert BERT embeddings to NumPy array
# # Ensure we only extract the [CLS] token embedding for each input, so we have a 2D array
# X_train_bert_cls = np.array([emb[0] for emb in X_train_bert])  # Extract the first embedding (CLS token)
# X_test_bert_cls = np.array([emb[0] for emb in X_test_bert])

# # Check the shapes to make sure both are 2D
# print(X_train_tfidf_dense.shape)  # (number of samples, number of features)
# print(X_train_bert_cls.shape)     # (number of samples, 768)

# # Combine dense TF-IDF features with the CLS token BERT embeddings
# X_train_combined = np.hstack([X_train_tfidf_dense, X_train_bert_cls])
# X_test_combined = np.hstack([X_test_tfidf_dense, X_test_bert_cls])


(11788, 768)
(11788, 768)


In [10]:
# # Train SVC model on the combined features
# svc_model = SVC(kernel='rbf', C=10)
# svc_model.fit(X_train_combined, y_train)

# # Predict on the test set
# y_pred_combined = svc_model.predict(X_test_combined)

# # Evaluate the model
# print(classification_report(y_test, y_pred_combined))
# #  precision    recall  f1-score   support

# #            0       0.52      0.55      0.53       770
# #            1       0.51      0.48      0.50       819
# #            2       0.66      0.61      0.64       671
# #            3       0.81      0.85      0.83       687

# #     accuracy                           0.62      2947
# #    macro avg       0.62      0.62      0.62      2947
# # weighted avg       0.61      0.62      0.62      2947

              precision    recall  f1-score   support

           0       0.51      0.54      0.53       770
           1       0.50      0.47      0.49       819
           2       0.64      0.61      0.62       671
           3       0.81      0.84      0.83       687

    accuracy                           0.61      2947
   macro avg       0.61      0.62      0.62      2947
weighted avg       0.61      0.61      0.61      2947



In [19]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from transformers import DistilBertTokenizer, DistilBertModel
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder

# # Check if CUDA is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Load the tokenizer and the DistilBERT model
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# distilbert_model = DistilBertModel.from_pretrained('distilbert-base-uncased', output_hidden_states=True).to(device)

# # Function to extract mean-pooled embeddings from DistilBERT
# def extract_mean_pooled_embeddings(text_data):
#     inputs = tokenizer(text_data, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
#     with torch.no_grad():
#         outputs = distilbert_model(**inputs)
#     last_hidden_state = outputs.last_hidden_state
#     # Mean pooling across all tokens
#     mean_pooled_embedding = torch.mean(last_hidden_state, dim=1)
#     return mean_pooled_embedding.cpu().numpy()

# # Split the dataset (based on your data structure)
# X_train, X_test, y_train, y_test = train_test_split(data['cleaned_lyrics'], data['mood_cats'], test_size=0.2, random_state=42)

# # One-hot encode the labels
# # One-hot encode the labels
# encoder = OneHotEncoder(sparse=False)
# y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))  # Convert Series to NumPy array
# y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1))  # Convert Series to NumPy array

# # Extract BERT embeddings for the training and testing sets
# X_train_bert = [extract_mean_pooled_embeddings(text) for text in X_train]
# X_test_bert = [extract_mean_pooled_embeddings(text) for text in X_test]

# # Convert to tensor and move to the appropriate device (GPU or CPU)
# X_train_bert = torch.tensor(X_train_bert).squeeze(1).to(device)
# y_train_tensor = torch.tensor(y_train_encoded).float().to(device)
# X_test_bert = torch.tensor(X_test_bert).squeeze(1).to(device)
# y_test_tensor = torch.tensor(y_test_encoded).float().to(device)

Using device: cuda


C:\Users\melih\AppData\Local\Temp\ipykernel_8904\2022592209.py:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  X_train_bert = torch.tensor(X_train_bert).squeeze(1).to(device)


In [20]:
# # Define the feed-forward neural network model
# class FFNN(nn.Module):
#     def __init__(self, input_size, num_classes):
#         super(FFNN, self).__init__()
#         self.fc1 = nn.Linear(input_size, 128)  # First layer with 128 neurons
#         self.fc2 = nn.Linear(128, 64)          # Second layer with 64 neurons
#         self.fc3 = nn.Linear(64, num_classes)  # Output layer
#         self.dropout = nn.Dropout(0.5)         # Dropout to prevent overfitting

#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = torch.relu(self.fc2(x))
#         x = self.dropout(x)
#         x = torch.softmax(self.fc3(x), dim=1)
#         return x
    
# # Instantiate the model and move it to the device (GPU or CPU)
# input_size = 768  # DistilBERT embedding size is 768
# num_classes = y_train_tensor.shape[1]  # Number of classes (from one-hot encoding)
# model = FFNN(input_size=input_size, num_classes=num_classes).to(device)

# # Define the loss function and optimizer
# criterion = nn.BCELoss()  # For multi-class classification with one-hot encoded labels
# optimizer = optim.Adam(model.parameters(), lr=1e-4)    

In [21]:
# # Training loop
# epochs = 10
# for epoch in range(epochs):
#     model.train()
#     optimizer.zero_grad()
#     outputs = model(X_train_bert)
#     loss = criterion(outputs, y_train_tensor)
#     loss.backward()
#     optimizer.step()
    
#     # Print training progress
#     print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# # Evaluate the model on the test set
# model.eval()
# with torch.no_grad():
#     outputs_test = model(X_test_bert)
#     predictions = torch.argmax(outputs_test, dim=1)
#     actuals = torch.argmax(y_test_tensor, dim=1)
    
#     accuracy = (predictions == actuals).float().mean().item()
#     print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/10, Loss: 0.5630422830581665
Epoch 2/10, Loss: 0.562897801399231
Epoch 3/10, Loss: 0.5626776218414307
Epoch 4/10, Loss: 0.5625957250595093
Epoch 5/10, Loss: 0.5620562434196472
Epoch 6/10, Loss: 0.5617476105690002
Epoch 7/10, Loss: 0.5615506768226624
Epoch 8/10, Loss: 0.5615678429603577
Epoch 9/10, Loss: 0.5613292455673218
Epoch 10/10, Loss: 0.5610941648483276
Test Accuracy: 30.23%


In [22]:
# # Evaluate the model on the test set
# model.eval()
# with torch.no_grad():
#     outputs_test = model(X_train_bert)
#     predictions = torch.argmax(outputs_test, dim=1)
#     actuals = torch.argmax(y_train_tensor, dim=1)
    
#     accuracy = (predictions == actuals).float().mean().item()
#     print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 30.96%
